In [1]:
#Installation
!pip install pandas
!pip install xlsxwriter
!pip install openpyxl
!pip install numpy

In [ ]:
#Imports
import io
from xlsxwriter.workbook import Workbook
import csv
import pandas as pd
import xlrd
from datetime import datetime
from datetime import date, timedelta
import random
import numpy as np
import openpyxl
import os
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font
from openpyxl.styles import Border, Side

In [ ]:
#Constants
WEEKDAYS={
    0:"M",
    1:"T",
    2:"W",
    3:"TH",
    4:"F",
    5:"S",
    6:"SU",
}
DAYS_31=['01','03','05','07','08','10','12']
DAYS_30=['04','06','09','11']
DAYS_29=['02']

HEADINGS={'P1':'MONTHLY STATUS REPORT',
          'P2':'19 FEB 2024 TO 26 FEB 2024',
          'A3':'COMPANY:MEPSC',
          'N3':'COLLEGE NAME: KSKMSKMM',
          'Z3':'PRINTED ON 1 MARCH'
         }

In [ ]:
#Inputs
MONTH = '2024-02'
DATE_FORMAT = '%Y-%m-%d'
START_DATE = '2024-02-19'
END_DATE = '2024-02-26'
HOLIDAYS = []


In [ ]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)+1):
        yield start_date + timedelta(n)


def distribute_attendance(present_days, total_days):
    present_days = present_days
    attendance = ['P'] * present_days + ['A'] * (total_days - present_days)
    random.shuffle(attendance)
    return attendance

In [ ]:
def total_possible_days(month,date_format):
  month_days=""
  available_days = []
  available_days.append('DAYS')
  if month[5:] in DAYS_31:
     month_days+="-31"
     for single_date in daterange(datetime.strptime(str(month)+"-01",date_format), datetime.strptime(str(month)+"-31",date_format)):
        available_days.append(str(single_date.day)+" "+WEEKDAYS[int(single_date.weekday())])
  if month[5:] in DAYS_30:
     month_days+="-30"
     for single_date in daterange(datetime.strptime(str(month)+"-01",date_format), datetime.strptime(str(month)+"-30",date_format)):
        available_days.append(str(single_date.day)+" "+WEEKDAYS[int(single_date.weekday())])
  if month[5:] in DAYS_29:
     month_days+="-29"
     for single_date in daterange(datetime.strptime(str(month)+"-01",date_format), datetime.strptime(str(month)+"-29",date_format)):
        available_days.append(str(single_date.day)+" "+WEEKDAYS[int(single_date.weekday())])
  return available_days

print(total_possible_days(MONTH,DATE_FORMAT))

['DAYS', '1 TH', '2 F', '3 S', '4 SU', '5 M', '6 T', '7 W', '8 TH', '9 F', '10 S', '11 SU', '12 M', '13 T', '14 W', '15 TH', '16 F', '17 S', '18 SU', '19 M', '20 T', '21 W', '22 TH', '23 F', '24 S', '25 SU', '26 M', '27 T', '28 W', '29 TH']


In [ ]:
def holiday_days():
  holiday_dates = []
  for i in HOLIDAYS:
    date=WEEKDAYS[datetime.strptime(i,DATE_FORMAT).weekday()]
    if date!='SU':
      holiday_dates.append(str(datetime.strptime(i,DATE_FORMAT).day)+" "+str(date))
  return holiday_dates

In [ ]:
def sundays():
  sunday_dates=[]
  for single_date in daterange(datetime.strptime(START_DATE,DATE_FORMAT), datetime.strptime(END_DATE,DATE_FORMAT)):
      if WEEKDAYS[int(single_date.weekday())]=='SU':
          sunday_dates.append(str(single_date.day)+" "+WEEKDAYS[int(single_date.weekday())])
  return sunday_dates

In [ ]:
def loadfile(file_path):
  data = pd.read_csv(file_path)
  return data

In [ ]:
def merge_rows_for_headings(empty_file,row_no):

  # Load the Excel file
  wb = load_workbook(empty_file)

  # Select the worksheet
  ws = wb['Sheet1']

  # Specify the row you want to merge
  row_to_merge = row_no

  # Get the number of columns in the worksheet
  num_columns = ws.max_column

  # Get the range of cells in the specified row
  row_range = ws[f'A{row_to_merge}:{get_column_letter(num_columns)}{row_to_merge}']

  # Join the values of all cells in the row into a single string
  merged_value = ' '.join([cell.value if cell.value else '' for cell in row_range[0]])

  # Clear the contents of the cells in the row
  for cell in row_range[0]:
      cell.value = None

  # Set the merged value in the first cell of the row
  ws.cell(row=row_to_merge, column=1, value=merged_value)

  # Save the modified Excel file
  wb.save('empty_input.xlsx')


In [ ]:
def fill_headings(file_path,text,position):
  wb = load_workbook(file_path)

  # Select the worksheet
  ws = wb['Sheet1']

  # Specify the cell coordinates where you want to insert the value
  cell_to_insert = position  # Example: insert value into cell A1

  # Insert the value into the specified cell
  value_to_insert = text
  ws[cell_to_insert] = value_to_insert
  bold_font = Font(bold=True)
  ws[cell_to_insert].font = bold_font

  # Save the modified Excel file
  wb.save(file_path)

In [ ]:
def add_borders(file_path,rows_index):
  wb = load_workbook(file_path)

  # Select the worksheet
  ws = wb['Sheet1']  # Replace 'Sheet1' with the name of your sheet

  # Specify the row for which you want to add borders
  for i in rows_index:
    row_index = i

    # Specify the border style (you can customize this as needed)
    border = Border(top=Side(style='thin'), bottom=Side(style='thin'), left=Side(style='thin'), right=Side(style='thin'))

    # Apply the border style to each cell in the specified row
    for cell in ws[row_index]:
        cell.border = border

    # Save the modified Excel file
    wb.save(file_path)

In [ ]:
def add_days_columns(file_path,available_days):
  wb = load_workbook(file_path)

  # Select the worksheet
  ws = wb['Sheet1']

  row_index = 6  # Example: insert elements into row 1

  # Define the list of elements to insert
  elements_list = available_days  # Example list of elements
  border = Border(top=Side(style='thin'), bottom=Side(style='thin'), left=Side(style='thin'), right=Side(style='thin'))
  # Iterate over the list and insert each element into the corresponding cell in the row
  for col_index, value in enumerate(elements_list, start=1):
      ws.cell(row=row_index, column=col_index, value=value).font=Font(bold=True)
      ws.cell(row=row_index, column=col_index).border = border

    # Save the modified Excel file
  wb.save(file_path)

In [ ]:
no_of_days=abs((datetime.strptime(END_DATE,DATE_FORMAT).date()-datetime.strptime(START_DATE,DATE_FORMAT).date()).days)+1
no_of_days=no_of_days-(len(holiday_days())+len(sundays()))
def input_for_a_student(file_path,index,code,name):

    present_days=random.randrange(int(no_of_days*0.9),int(no_of_days))
    present_days_list=distribute_attendance(present_days,no_of_days)

    # Name and Code row
    wb = load_workbook(file_path)
    ws = wb['Sheet1']
    cell_to_insert = "A"+str(index)

    value_to_insert = "EMP CODE:"+str(code)+"                 EMP NAME:"+str(name)
    ws[cell_to_insert] = value_to_insert

    #Status
    ws = wb['Sheet1']
    cell_to_insert = "A"+str(index+1)
    border = Border(top=Side(style='thin'), bottom=Side(style='thin'), left=Side(style='thin'), right=Side(style='thin'))

    # Apply the border style to each cell in the specified row
    for cell in ws[index+1]:
        cell.border = border
    value_to_insert = "Status"
    ws[cell_to_insert] = value_to_insert
    ws[cell_to_insert].font = Font(bold=True)

    #In time
    ws = wb['Sheet1']
    cell_to_insert = "A"+str(index+2)
    border = Border(top=Side(style='thin'), bottom=Side(style='thin'), left=Side(style='thin'), right=Side(style='thin'))

    # Apply the border style to each cell in the specified row
    for cell in ws[index+2]:
        cell.border = border
    value_to_insert = "In Time"
    ws[cell_to_insert] = value_to_insert
    ws[cell_to_insert].font = Font(bold=True)

    #Outtime

    ws = wb['Sheet1']
    cell_to_insert = "A"+str(index+3)
    border = Border(top=Side(style='thin'), bottom=Side(style='thin'), left=Side(style='thin'), right=Side(style='thin'))

    # Apply the border style to each cell in the specified row
    for cell in ws[index+3]:
        cell.border = border
    value_to_insert = "Out Time"
    ws[cell_to_insert] = value_to_insert
    ws[cell_to_insert].font = Font(bold=True)

    #Total
    ws = wb['Sheet1']
    cell_to_insert = "A"+str(index+4)
    border = Border(top=Side(style='thin'), bottom=Side(style='thin'), left=Side(style='thin'), right=Side(style='thin'))

    # Apply the border style to each cell in the specified row
    for cell in ws[index+4]:
        cell.border = border
    value_to_insert = "Total"
    ws[cell_to_insert] = value_to_insert
    ws[cell_to_insert].font = Font(bold=True)

    wb.save('new.xlsx')
    return present_days_list



In [ ]:
if __name__=="__main__":
  df = pd.DataFrame(columns=['A','B','C','D','E'])
  merge_rows_for_headings('/content/empty_input.xlsx',1)
  merge_rows_for_headings('/content/empty_input.xlsx',2)
  merge_rows_for_headings('/content/empty_input.xlsx',3)
  merge_rows_for_headings('/content/empty_input.xlsx',4)

  for i in HEADINGS:
    fill_headings('/content/empty_input.xlsx',HEADINGS[i],i)
  available_days = total_possible_days(MONTH,DATE_FORMAT)
  add_days_columns('/content/empty_input.xlsx',available_days)
  input_for_a_student('/content/empty_input.xlsx',8,1,3)


